In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import fake_useragent
import re
import warnings
warnings.filterwarnings("ignore")

ua = fake_useragent.UserAgent()

In [2]:
def get_url(category, tot_pages):
    for page in range(1,tot_pages+1):
        if page % log_frequency == 0:
            print(f'Log: working over page {page}...')       

        url_main = f"https://mir-sporta.com/category/{category}/?PAGEN_1={page}"
        response = requests.get(url_main, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')
        data = soup.find_all(class_="thumb")
        for item in data:
            item_url = 'https://mir-sporta.com' + item.attrs['href']
            yield item_url

In [12]:
def get_item(cat_gen, category, tot_pages):
    count = 0
    for url_item in get_url(category, tot_pages):
        count += 1
        response = requests.get(url_item, headers = {'user-agent': ua.random})
        soup = BeautifulSoup(response.content, 'lxml')

        try:
            title = soup.find('h1').text
        except:
            title = ''

        try:
            article = soup.find(class_ = 'c-value').text
            article = article.replace(' ', '').replace('\n', '')
            article = article.replace('Артикул:', '')
        except:
            article = ''

        try:
            image = soup.find(class_ = 'l-image-box l-image-box_fill', href = True)
            url = image['href']
            image_ref = f"images/{cat_gen}/{count}_mirsporta_{article}.jpeg"
            with open(f"images/{cat_gen}/{count}_mirsporta_{article}.jpeg", "wb") as f:
                r = requests.get(url)
                f.write(r.content)

        except:
            print(f'Log: no image for {article} found...')
            url = ''
            image_ref = ''

        try:
            desc = soup.find('h2').next_sibling.text
            desc = desc.replace('\n', ' ')
        except:
            desc = ''

        try:
            char = soup.find(class_ = 'c-product-features c-product-features_2_cols').text
            char = re.sub(r"(?<=\w)([А-ЯA-Z])", r" \1", char)
            char = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", char).strip()
            char = char.replace('Основные', '').replace('Примечание Технические характеристики, описание, комплектация поставки и внешний вид товаров не является публичной офертой. Подробную информацию уточняйте у наших менеджеров.', '')
        except:
            char = ''

        try:
            price_lst = []
            price_info = soup.find(class_ = 'c-product-add-to-cart__prices')
            for price in price_info:
                price = price.text.replace('₽', '').replace(' ', '')
                price_lst.append(price)
            price_fin = price_lst[0]

        except:
            price = ''

        try:
            cat = soup.find(class_ = 'c-breadcrumbs c-breadcrumbs_current-item-mode_show').text
            temp_str = re.sub(re.compile('[^а-яА-Я ]'), '', cat)
            lst = re.findall('[А-Я][^А-Я]*', temp_str)

            try:
                cat_1 = lst[-3]
            except:
                cat_1 = ''
            try:
                cat_2 = lst[-2]
            except:
                cat_2 = ''
            try:
                cat_3 = re.sub(re.compile('[^a-zA-Z ]'), '', cat).strip().split(' ')[0]
            except:
                cat_3 = ''
        except:
            cat_1 = ''
            cat_2 = ''
            cat_3 = ''

        yield title, article, price_fin, cat_1, cat_2, cat_3, url, desc, char, image_ref

In [4]:
df_columns = [
    "title",
    "article",
    "price",
    "cat_1",
    "cat_2",
    "cat_3",
    "url",
    "description",
    "caracteristics",
    "img_ref",
]

df = pd.DataFrame(columns = df_columns)
save_frequency = 200
log_frequency = 8

In [5]:
import time

def to_csv(cat_gen, category, tot_pages):
    counter = 0  

    for item in get_item(cat_gen, category, tot_pages):
            data = []
            for i in range(len(df_columns)):
                data.append(item[i])
            df.loc[len(df.index)] = data
            time.sleep(0.05)

            counter += 1
            if counter % save_frequency == 0:
                df.to_csv(f'{cat_gen}_mirsporta.csv')
                print(f'{counter} items saved in csv..')
    df.to_csv(f'{cat_gen}_mirsporta.csv')

    print(f'\nTotal items saved: {counter}')  

In [6]:
to_csv('begovye_dorozhki','begovye-dorozhki', 15)

Log: working over page 8...

Total items saved: 0
